# Анализ данных NGS. Домашнее задание № 4

Выполнил: Олег Вавулов

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import gzip
sys.path.insert(0, os.getcwd())
import functions as f
import pysam
from tqdm import tqdm
import pandas as pd

Данные храним вне проекта

In [2]:
cwd = os.getcwd()
data_path = os.path.join(f.get_prev_path(cwd, 2), "data", "hw_4")
bin_path = os.path.join(f.get_prev_path(cwd, 2), "bin")

In [3]:
if not os.path.exists("original"):
    os.mkdir("original")
if not os.path.exists("trimmomatic"):
    os.mkdir("trimmomatic")
if not os.path.exists("bayeshammer"):
    os.mkdir("bayeshammer")
if not os.path.exists(os.path.join(data_path, "trimmomatic")):
    os.mkdir(os.path.join(data_path, "trimmomatic"))
if not os.path.exists(os.path.join(data_path, "bayeshammer")):
    os.mkdir(os.path.join(data_path, "bayeshammer"))

In [4]:
with open(os.path.join(data_path, "MG1655-K12.first10K.fasta")) as f:
    reference = f.read()
reference = "".join(reference.split("\n")[1:])
print(f"Reference genome length: {len(reference)}")

Reference genome length: 10000


In [5]:
def analyse_mapping(mapping, report):
    if mapping.is_secondary:
        return
    r = reference[mapping.reference_start: mapping.reference_end]
    q = mapping.seq[mapping.qstart: mapping.qend]
    errors_string = ""
    r_idx = 0
    q_idx = 0
    for sort, n in mapping.cigar:
        if sort == 0:
            for _ in range(n):
                r_nt = r[r_idx]
                q_nt = q[q_idx]
                errors_string += "E" if r_nt != q_nt else "C"
                r_idx += 1
                q_idx += 1
        elif sort == 1:
            errors_string += n * "E"
            q_idx += n
        elif sort == 2:
            r_idx += n
        elif sort == 4 or sort == 5:
            pass
        else:
            raise Exception
    key = mapping.qname + ("_read1" if mapping.is_read1 else "_read2")
    report[key] = errors_string, mapping.pos

In [7]:
def analyse_mapping(mapping, report):
    if mapping.is_secondary:
        return
    r = reference[mapping.reference_start: mapping.reference_end]
    q = mapping.seq[mapping.qstart: mapping.qend]
    errors_string = ""
    r_idx = 0
    q_idx = 0
    for sort, n in mapping.cigar:
        if sort == 0:
            for _ in range(n):
                r_nt = r[r_idx]
                q_nt = q[q_idx]
                errors_string += "E" if r_nt != q_nt and q_nt != "N" else "C"
                r_idx += 1
                q_idx += 1
        elif sort == 1:
            errors_string += n * "E"
            q_idx += n
        elif sort == 2:
            r_idx += n
        elif sort == 4 or sort == 5:
            pass
        else:
            raise Exception
    key = mapping.qname + ("_read1" if mapping.is_read1 else "_read2")
    report[key] = errors_string, mapping.pos

# Оригинальные риды

In [6]:
!fastqc -o original ../../data/hw_4/ecoli_10K_*

Started analysis of ecoli_10K_err_1.fastq
Approx 5% complete for ecoli_10K_err_1.fastq
Approx 10% complete for ecoli_10K_err_1.fastq
Approx 15% complete for ecoli_10K_err_1.fastq
Approx 20% complete for ecoli_10K_err_1.fastq
Approx 25% complete for ecoli_10K_err_1.fastq
Approx 30% complete for ecoli_10K_err_1.fastq
Approx 35% complete for ecoli_10K_err_1.fastq
Approx 40% complete for ecoli_10K_err_1.fastq
Approx 45% complete for ecoli_10K_err_1.fastq
Approx 50% complete for ecoli_10K_err_1.fastq
Approx 55% complete for ecoli_10K_err_1.fastq
Approx 60% complete for ecoli_10K_err_1.fastq
Approx 65% complete for ecoli_10K_err_1.fastq
Approx 70% complete for ecoli_10K_err_1.fastq
Approx 75% complete for ecoli_10K_err_1.fastq
Approx 80% complete for ecoli_10K_err_1.fastq
Approx 85% complete for ecoli_10K_err_1.fastq
Approx 90% complete for ecoli_10K_err_1.fastq
Approx 95% complete for ecoli_10K_err_1.fastq
Analysis complete for ecoli_10K_err_1.fastq
Started analysis of ecoli_10K_err_2.fastq

Качество для оригинальных ридов (FastQC)

<img src="original/orig_qual.png" width=600 height=600 />

In [7]:
!bwa index ../../data/hw_4/MG1655-K12.first10K.fasta

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index ../../data/hw_4/MG1655-K12.first10K.fasta
[main] Real time: 0.013 sec; CPU: 0.013 sec


In [8]:
!bwa mem ../../data/hw_4/MG1655-K12.first10K.fasta \
../../data/hw_4/ecoli_10K_err_1.fastq ../../data/hw_4/ecoli_10K_err_2.fastq \
> ../../data/hw_4/alignment.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 59278 sequences (5927800 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 29468, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.40, 10.18)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 59278 reads in 1.308 CPU sec, 1.310 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem ../../data/hw_4/MG1655-K12.first10K.fasta ../../data/hw_4/ecoli_10K_err_1.fastq ../../data/hw_4/ecoli_10K_err_2.fastq
[main] Real time: 1.413 sec; CPU: 1.414 sec


In [9]:
!samtools flagstat ../../data/hw_4/alignment.sam

59278 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
59261 + 0 mapped (99.97% : N/A)
59278 + 0 paired in sequencing
29639 + 0 read1
29639 + 0 read2
59216 + 0 properly paired (99.90% : N/A)
59246 + 0 with itself and mate mapped
15 + 0 singletons (0.03% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [10]:
original_reads_errors = {}
with pysam.AlignmentFile(os.path.join(data_path, "alignment.sam")) as f:
    for mapping in tqdm(f.fetch(), total=59278):
        analyse_mapping(mapping, original_reads_errors)

100%|██████████| 59278/59278 [00:01<00:00, 54765.39it/s]


# Trimmomatic

In [11]:
!trimmomatic PE -phred33 -trimlog ./trimmomatic/trimlog.txt -summary ./trimmomatic/summary.txt \
../../data/hw_4/ecoli_10K_err_1.fastq ../../data/hw_4/ecoli_10K_err_2.fastq \
../../data/hw_4/trimmomatic/corrected_1P.fq ../../data/hw_4/trimmomatic/corrected_1U.fq \
../../data/hw_4/trimmomatic/corrected_2P.fq ../../data/hw_4/trimmomatic/corrected_2U.fq \
LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:20

TrimmomaticPE: Started with arguments:
 -phred33 -trimlog ./trimmomatic/trimlog.txt -summary ./trimmomatic/summary.txt ../../data/hw_4/ecoli_10K_err_1.fastq ../../data/hw_4/ecoli_10K_err_2.fastq ../../data/hw_4/trimmomatic/corrected_1P.fq ../../data/hw_4/trimmomatic/corrected_1U.fq ../../data/hw_4/trimmomatic/corrected_2P.fq ../../data/hw_4/trimmomatic/corrected_2U.fq LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:20
Input Read Pairs: 29639 Both Surviving: 28722 (96,91%) Forward Only Surviving: 597 (2,01%) Reverse Only Surviving: 276 (0,93%) Dropped: 44 (0,15%)
TrimmomaticPE: Completed successfully


In [12]:
!fastqc -o trimmomatic ../../data/hw_4/trimmomatic/corrected_1P.fq ../../data/hw_4/trimmomatic/corrected_2P.fq

Started analysis of corrected_1P.fq
Approx 5% complete for corrected_1P.fq
Approx 10% complete for corrected_1P.fq
Approx 15% complete for corrected_1P.fq
Approx 20% complete for corrected_1P.fq
Approx 25% complete for corrected_1P.fq
Approx 30% complete for corrected_1P.fq
Approx 35% complete for corrected_1P.fq
Approx 40% complete for corrected_1P.fq
Approx 45% complete for corrected_1P.fq
Approx 50% complete for corrected_1P.fq
Approx 55% complete for corrected_1P.fq
Approx 60% complete for corrected_1P.fq
Approx 65% complete for corrected_1P.fq
Approx 70% complete for corrected_1P.fq
Approx 75% complete for corrected_1P.fq
Approx 80% complete for corrected_1P.fq
Approx 85% complete for corrected_1P.fq
Approx 90% complete for corrected_1P.fq
Approx 95% complete for corrected_1P.fq
Analysis complete for corrected_1P.fq
Started analysis of corrected_2P.fq
Approx 5% complete for corrected_2P.fq
Approx 10% complete for corrected_2P.fq
Approx 15% complete for corrected_2P.fq
Approx 20% c

Качество для ридов, обработанных Trimmomatic (FastQC)

<img src="trimmomatic/trim_qual.png" width=600 height=600 />

In [13]:
!bwa mem ../../data/hw_4/MG1655-K12.first10K.fasta \
../../data/hw_4/trimmomatic/corrected_1P.fq ../../data/hw_4/trimmomatic/corrected_2P.fq \
> ../../data/hw_4/trimmomatic/alignment.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 57444 sequences (5119460 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 28434, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.38, 10.19)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 57444 reads in 0.937 CPU sec, 0.938 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem ../../data/hw_4/MG1655-K12.first10K.fasta ../../data/hw_4/trimmomatic/corrected_1P.fq ../../data/hw_4/trimmomatic/corrected_2P.fq
[main] Real time: 1.047 sec; CPU: 1.0

In [14]:
!samtools flagstat ../../data/hw_4/trimmomatic/alignment.sam

57444 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
57419 + 0 mapped (99.96% : N/A)
57444 + 0 paired in sequencing
28722 + 0 read1
28722 + 0 read2
57364 + 0 properly paired (99.86% : N/A)
57394 + 0 with itself and mate mapped
25 + 0 singletons (0.04% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [9]:
corrected_reads_errors = {}
with pysam.AlignmentFile(os.path.join(data_path, "trimmomatic", "alignment.sam")) as f:
    for mapping in tqdm(f.fetch(), total=57444):
        analyse_mapping(mapping, corrected_reads_errors)

100%|██████████| 57444/57444 [00:01<00:00, 55253.26it/s]


In [10]:
correction_matrix = pd.DataFrame(
    index=["Error (Original)", "Correct (Original)"],
    columns=["Error (Corrected)", "Correct (Corrected)", "Absence (Corrected)"],
    data=0
)

In [11]:
for qname, (orig_seq, orig_pos) in tqdm(original_reads_errors.items()):
    try: # проверяем сохранилась ли пара прочтений после коррекции
        corr_seq, corr_pos = corrected_reads_errors[qname]
    except KeyError:
        continue
    # дописываем плейсхолдеры в начало и конец обрезанных ридов
    if orig_pos != corr_pos:
        corr_seq = abs(orig_pos-corr_pos) * "A" + corr_seq
    if len(orig_seq) > len(corr_seq):
        corr_seq += abs(len(orig_seq) - len(corr_seq)) * "A"
    elif len(orig_seq) < len(corr_seq): # так как при выравнивании оригинальный рид тоже обрезается (softclipping), \
                                        # случается так, что он оказывается короче скорректированного рида
        orig_seq += abs(len(orig_seq) - len(corr_seq)) * "A"
    assert len(orig_seq) == len(corr_seq)
    for orig_nt, corr_nt in zip(list(orig_seq), list(corr_seq)):
        if orig_nt == "C" and corr_nt == "E":
            correction_matrix.loc["Correct (Original)", "Error (Corrected)"] += 1
        elif orig_nt == "C" and corr_nt == "C":
            correction_matrix.loc["Correct (Original)", "Correct (Corrected)"] += 1
        elif orig_nt == "C" and corr_nt == "A":
            correction_matrix.loc["Correct (Original)", "Absence (Corrected)"] += 1
        elif orig_nt == "E" and corr_nt == "E":
            correction_matrix.loc["Error (Original)", "Error (Corrected)"] += 1
        elif orig_nt == "E" and corr_nt == "C":
            correction_matrix.loc["Error (Original)", "Correct (Corrected)"] += 1
        elif orig_nt == "E" and corr_nt == "A":
            correction_matrix.loc["Error (Original)", "Absence (Corrected)"] += 1        

100%|██████████| 59278/59278 [16:44<00:00, 59.00it/s]


In [12]:
correction_matrix

,Error (Corrected),Correct (Corrected),Absence (Corrected)
Error (Original),15156,131,31816
Correct (Original),368,5098560,421938


Old

In [18]:
correction_matrix

,Error (Corrected),Correct (Corrected),Absence (Corrected)
Error (Original),17848,135,32592
Correct (Original),374,5095858,421162


# BayesHammer

In [19]:
!../../../../bin/SPAdes-3.14.1-Darwin/bin/spades.py --only-error-correction\
-1 ../../data/hw_4/ecoli_10K_err_1.fastq -2 ../../data/hw_4/ecoli_10K_err_2.fastq \
-o ../../data/hw_4/bayeshammer



== Warning ==  output dir is not empty! Please, clean output directory before run.




== Warning ==  No assembly mode was specified! If you intend to assemble high-coverage multi-cell/isolate data, use '--isolate' option.


Command line: ../../../../bin/SPAdes-3.14.1-Darwin/bin/spades.py	--only-error-correction	-1	/Users/a18264698/Desktop/BIOINF/2_semester/NGS/data/hw_4/ecoli_10K_err_1.fastq	-2	/Users/a18264698/Desktop/BIOINF/2_semester/NGS/data/hw_4/ecoli_10K_err_2.fastq	-o	/Users/a18264698/Desktop/BIOINF/2_semester/NGS/data/hw_4/bayeshammer	

System information:
  SPAdes version: 3.14.1
  Python version: 3.7.4
  OS: Darwin-19.6.0-x86_64-i386-64bit

Output dir: /Users/a18264698/Desktop/BIOINF/2_semester/NGS/data/hw_4/bayeshammer
Mode: ONLY read error correction (without assembling)
Debug mode is turned OFF

Dataset parameters:
  Standard mode
  For multi-cell/isolate data we recommend to use '--isolate' option; for single-cell MDA data use '--sc'; for metagenomic data use '--meta';

In [20]:
!gzip -dc ../../data/hw_4/bayeshammer/corrected/ecoli_10K_err_1.00.0_0.cor.fastq.gz \
> ../../data/hw_4/bayeshammer/corrected_1P.fq
!gzip -dc ../../data/hw_4/bayeshammer/corrected/ecoli_10K_err_2.00.0_0.cor.fastq.gz \
> ../../data/hw_4/bayeshammer/corrected_2P.fq

In [21]:
!fastqc -o bayeshammer ../../data/hw_4/bayeshammer/corrected_1P.fq ../../data/hw_4/bayeshammer/corrected_2P.fq

Started analysis of corrected_1P.fq
Approx 5% complete for corrected_1P.fq
Approx 10% complete for corrected_1P.fq
Approx 15% complete for corrected_1P.fq
Approx 20% complete for corrected_1P.fq
Approx 25% complete for corrected_1P.fq
Approx 30% complete for corrected_1P.fq
Approx 35% complete for corrected_1P.fq
Approx 40% complete for corrected_1P.fq
Approx 45% complete for corrected_1P.fq
Approx 50% complete for corrected_1P.fq
Approx 55% complete for corrected_1P.fq
Approx 60% complete for corrected_1P.fq
Approx 65% complete for corrected_1P.fq
Approx 70% complete for corrected_1P.fq
Approx 75% complete for corrected_1P.fq
Approx 80% complete for corrected_1P.fq
Approx 85% complete for corrected_1P.fq
Approx 90% complete for corrected_1P.fq
Approx 95% complete for corrected_1P.fq
Analysis complete for corrected_1P.fq
Started analysis of corrected_2P.fq
Approx 5% complete for corrected_2P.fq
Approx 10% complete for corrected_2P.fq
Approx 15% complete for corrected_2P.fq
Approx 20% c

Качество для ридов, обработанных BayesHammer (FastQC)

<img src="bayeshammer/bayes_qual.png" width=600 height=600 />

In [22]:
!bwa mem ../../data/hw_4/MG1655-K12.first10K.fasta \
../../data/hw_4/bayeshammer/corrected_1P.fq ../../data/hw_4/bayeshammer/corrected_2P.fq \
> ../../data/hw_4/bayeshammer/alignment.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 57986 sequences (5798600 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 28895, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.39, 10.18)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 57986 reads in 0.887 CPU sec, 0.887 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem ../../data/hw_4/MG1655-K12.first10K.fasta ../../data/hw_4/bayeshammer/corrected_1P.fq ../../data/hw_4/bayeshammer/corrected_2P.fq
[main] Real time: 1.020 sec; CPU: 1.0

In [23]:
!samtools flagstat ../../data/hw_4/bayeshammer/alignment.sam

57986 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
57984 + 0 mapped (100.00% : N/A)
57986 + 0 paired in sequencing
28993 + 0 read1
28993 + 0 read2
57952 + 0 properly paired (99.94% : N/A)
57982 + 0 with itself and mate mapped
2 + 0 singletons (0.00% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [13]:
corrected_reads_errors = {}
with pysam.AlignmentFile(os.path.join(data_path, "bayeshammer", "alignment.sam")) as f:
    for mapping in tqdm(f.fetch(), total=57986):
        analyse_mapping(mapping, corrected_reads_errors)

100%|██████████| 57986/57986 [00:01<00:00, 55862.82it/s]


In [14]:
correction_matrix = pd.DataFrame(
    index=["Error (Original)", "Correct (Original)"],
    columns=["Error (Corrected)", "Correct (Corrected)", "Absence (Corrected)"],
    data=0
)

In [15]:
for qname, (orig_seq, orig_pos) in tqdm(original_reads_errors.items()):
    try: # проверяем сохранилась ли пара прочтений после коррекции
        corr_seq, corr_pos = corrected_reads_errors[qname]
    except KeyError:
        continue
    # дописываем плейсхолдеры в начало и конец обрезанных ридов
    if orig_pos != corr_pos:
        corr_seq = abs(orig_pos-corr_pos) * "A" + corr_seq
    if len(orig_seq) > len(corr_seq):
        corr_seq += abs(len(orig_seq) - len(corr_seq)) * "A"
    elif len(orig_seq) < len(corr_seq): # так как при выравнивании оригинальный рид тоже обрезается (softclipping), \
                                        # случается так, что он оказывается короче скорректированного рида
        orig_seq += abs(len(orig_seq) - len(corr_seq)) * "A"
    assert len(orig_seq) == len(corr_seq)
    for orig_nt, corr_nt in zip(list(orig_seq), list(corr_seq)):
        if orig_nt == "C" and corr_nt == "E":
            correction_matrix.loc["Correct (Original)", "Error (Corrected)"] += 1
        elif orig_nt == "C" and corr_nt == "C":
            correction_matrix.loc["Correct (Original)", "Correct (Corrected)"] += 1
        elif orig_nt == "C" and corr_nt == "A":
            correction_matrix.loc["Correct (Original)", "Absence (Corrected)"] += 1
        elif orig_nt == "E" and corr_nt == "E":
            correction_matrix.loc["Error (Original)", "Error (Corrected)"] += 1
        elif orig_nt == "E" and corr_nt == "C":
            correction_matrix.loc["Error (Original)", "Correct (Corrected)"] += 1
        elif orig_nt == "E" and corr_nt == "A":
            correction_matrix.loc["Error (Original)", "Absence (Corrected)"] += 1        

100%|██████████| 59278/59278 [16:40<00:00, 59.26it/s]


In [16]:
correction_matrix

,Error (Corrected),Correct (Corrected),Absence (Corrected)
Error (Original),2689,16792,28577
Correct (Original),125,5231939,341331


Old

In [27]:
correction_matrix

,Error (Corrected),Correct (Corrected),Absence (Corrected)
Error (Original),3575,18837,29158
Correct (Original),8341,5220792,340750
